## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now() - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=6)

def format_date(date):
    day = date.day
    # Months 3-7 (March–July) should be full; all others abbreviated with a period
    if date.month in (3, 4, 5, 6, 7):
        month_str = date.strftime("%B")      # e.g. "March", "April", ...
    else:
        month_str = date.strftime("%b") + "."  # e.g. "Jan.", "Feb.", "Aug.", ...

    return f"{month_str} {day}"


# Format both dates
week_ending = format_date(today)
week_starting = format_date(one_week_ago)

print(week_starting, week_ending)

June 30 July 6


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,June-30-2025,Condo/Co-op,1153 Cambridge G,Deerfield Beach,FL,33442.0,80000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/1153...,Beaches MLS,RX-11059318,N,Y,26.313327,-80.135436


In [8]:
print('Input county name:')
county = input()

Input county name:


 Broward


In [9]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [11]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [10]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [11]:
df2 = df[(df['SOLD DATE'] >= one_week_ago) & (df['SOLD DATE'] < today)]

In [12]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-07-01    28
2025-07-03    20
2025-07-02    16
2025-07-04     2
Name: count, dtype: int64

In [13]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [14]:
df_top_ten = df2.head(10)

In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [15]:
def process_response(url):
    try:
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"
        querystring = {"url": url}
        
        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }
        response = requests.get(base_url, headers=headers, params=querystring)
        response.raise_for_status()
        data = response.json()

        # Safely drill down, falling back to {} if any level is None or missing
        info = data.get('data') or {}
        panel = info.get('mainHouseInfoPanelInfo') or {}
        house = panel.get('mainHouseInfo') or {}

        # Extract listing agents
        listing_agents = house.get('listingAgents') or []
        list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName') if len(listing_agents) > 0 else None
        list_broker_name_1 = listing_agents[0].get('brokerName')           if len(listing_agents) > 0 else None
        list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName') if len(listing_agents) > 1 else None
        list_broker_name_2 = listing_agents[1].get('brokerName')           if len(listing_agents) > 1 else None

        # Extract buying agents
        buying_agents = house.get('buyingAgents') or []
        buy_agent_name_1  = buying_agents[0].get('agentInfo', {}).get('agentName') if len(buying_agents) > 0 else None
        buy_broker_name_1 = buying_agents[0].get('brokerName')             if len(buying_agents) > 0 else None
        buy_agent_name_2  = buying_agents[1].get('agentInfo', {}).get('agentName') if len(buying_agents) > 1 else None
        buy_broker_name_2 = buying_agents[1].get('brokerName')             if len(buying_agents) > 1 else None

        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return { key: None for key in [
            'url', 'list_agent_name_1','list_broker_name_1',
            'list_agent_name_2','list_broker_name_2',
            'buy_agent_name_1','buy_broker_name_1',
            'buy_agent_name_2','buy_broker_name_2'
        ] }

In [16]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [17]:
response_df = pd.DataFrame(response_list)

In [18]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [19]:
df_top_ten = merged_df

In [20]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-07-04,Condo/Co-op,730 N Ocean Blvd #803,Pompano Beach,FL,33062.0,2100000.0,2.0,2.5,...,-80.087321,https://www.redfin.com/FL/Pompano-Beach/730-N-...,David Marder,"Compass Florida, LLC",None,None,David Marder,"Compass Florida, LLC",None,None
1,PAST SALE,2025-07-01,Condo/Co-op,2711 S Ocean Dr #2701,Hollywood,FL,33019.0,1740000.0,3.0,3.5,...,-80.117509,https://www.redfin.com/FL/Hollywood/2711-S-Oce...,Jesus Rincon,Brokernation Real Estate Doral,None,None,Vivian Toppe,Borges Realty Group LLC,None,None
2,PAST SALE,2025-07-01,Condo/Co-op,1912 S Ocean Dr Unit 17A,Hallandale Beach,FL,33009.0,1065000.0,2.0,2.5,...,-80.118894,https://www.redfin.com/FL/Hallandale-Beach/191...,Pamela DiMartino,Coldwell Banker Realty,None,None,Iuliia Zohar,Best of Luxury Realty Corp.,None,None
3,PAST SALE,2025-07-03,Condo/Co-op,1504 Passion Vine Cir Unit 23-4,Weston,FL,33326.0,800000.0,4.0,2.5,...,-80.383006,https://www.redfin.com/FL/Weston/1504-Passion-...,Luis Vacca,Sarah Real Estate Group LLC,None,None,Maurizio Bettoli,"Bettoli Real Estate, LLC",None,None
4,PAST SALE,2025-07-02,Condo/Co-op,401 N Birch Rd #507,Fort Lauderdale,FL,33304.0,775000.0,1.0,1.5,...,-80.106224,https://www.redfin.com/FL/Fort-Lauderdale/401-...,None,None,None,None,None,None,None,None
5,PAST SALE,2025-07-02,Condo/Co-op,2880 NE 14th Street Cswy #804,Pompano Beach,FL,33062.0,700000.0,3.0,3.0,...,-80.091423,https://www.redfin.com/FL/Pompano-Beach/2880-N...,Stuart Goodwin,Century 21 Hansen Realty Inc,None,None,Eslin Guice,Icon Realty Advisors LLC,None,None
6,PAST SALE,2025-07-02,Condo/Co-op,1750 S Ocean Ln #317,Fort Lauderdale,FL,33316.0,490000.0,2.0,2.0,...,-80.106975,https://www.redfin.com/FL/Fort-Lauderdale/1750...,Rosina Lenardson,Lago Mar Realty Inc,None,None,Steven Bate,Rock and Roll Real Estate,None,None
7,PAST SALE,2025-07-02,Condo/Co-op,1945 S Ocean Dr #1512,Hallandale Beach,FL,33009.0,485000.0,1.0,1.5,...,-80.120771,https://www.redfin.com/FL/Hallandale-Beach/194...,Yure Contreras,"Brickell Realty Group, LLC",Juan Ramirez,"Brickell Realty Group, LLC",Mariana Martynov,Golden Keys Inc,None,None
8,PAST SALE,2025-07-01,Condo/Co-op,100 N Federal Hwy #1520,Fort Lauderdale,FL,33301.0,482000.0,2.0,2.0,...,-80.136665,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Adam Curtis,LoKation,None,None,Joseph Mongillo,LoKation,None,None
9,PAST SALE,2025-07-03,Condo/Co-op,110 N Federal Hwy #1403,Fort Lauderdale,FL,33301.0,470000.0,2.0,2.0,...,-80.136780,https://www.redfin.com/FL/Fort-Lauderdale/110-...,Patricia Finley,Coldwell Banker Realty,Justine Schade,Coldwell Banker Realty,None,One Sotheby's Int'l Realty,None,None


## Current Week's Values

In [21]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

66


In [22]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$322,508


In [23]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$270


In [24]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$21,285,500


In [25]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0           66              322508          270                     21285500.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [26]:
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input Previous Week Condo Sales Total (number units sold):
65
-----------
Input Previous Week Condo Average Sales Price:
326772
-----------
Input Previous Week Condo Average PSF:
264
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
21240150.0


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [27]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [28]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [29]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [30]:
pd.set_option('display.max_columns',None)

In [31]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-07-04,Condo/Co-op,730 N Ocean Blvd #803,Pompano Beach,FL,33062.0,2100000.0,2.0,2.5,Sabbia Beach,1880.0,NaN,2019.0,NaN,1117.0,2919.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/730-N-...,Beaches MLS,F10513197,N,Y,26.243932,-80.087321,https://www.redfin.com/FL/Pompano-Beach/730-N-...,David Marder,"Compass Florida, LLC",None,None,David Marder,"Compass Florida, LLC",None,None,1,orange


In [32]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [35]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-07-04,Condo/Co-op,730 N Ocean Blvd #803,Pompano Beach,FL,33062.0,2100000.0,2.0,2.5,Sabbia Beach,1880.0,NaN,2019.0,NaN,1117.0,2919.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/730-N-...,Beaches MLS,F10513197,N,Y,26.243932,-80.087321,https://www.redfin.com/FL/Pompano-Beach/730-N-...,David Marder,"Compass Florida, LLC",None,None,David Marder,"Compass Florida, LLC",None,None,1,orange
1,PAST SALE,2025-07-01,Condo/Co-op,2711 S Ocean Dr #2701,Hollywood,FL,33019.0,1740000.0,3.0,3.5,2711 HOLLYWOOD BEACH COND,2088.0,NaN,2009.0,NaN,833.0,2131.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/2711-S-Oce...,MARMLS,A11704214,N,Y,25.996265,-80.117509,https://www.redfin.com/FL/Hollywood/2711-S-Oce...,Jesus Rincon,Brokernation Real Estate Doral,None,None,Vivian Toppe,Borges Realty Group LLC,None,None,2,blue
2,PAST SALE,2025-07-01,Condo/Co-op,1912 S Ocean Dr Unit 17A,Hallandale Beach,FL,33009.0,1065000.0,2.0,2.5,MALAGA TOWERS CONDO,2305.0,NaN,1975.0,NaN,462.0,1895.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/191...,MARMLS,A11709779,N,Y,25.981695,-80.118894,https://www.redfin.com/FL/Hallandale-Beach/191...,Pamela DiMartino,Coldwell Banker Realty,None,None,Iuliia Zohar,Best of Luxury Realty Corp.,None,None,3,blue
3,PAST SALE,2025-07-03,Condo/Co-op,1504 Passion Vine Cir Unit 23-4,Weston,FL,33326.0,800000.0,4.0,2.5,MARIPOSA POINTE AT WESTON,2340.0,NaN,2002.0,NaN,342.0,960.0,Sold,NaN,NaN,https://www.redfin.com/FL/Weston/1504-Passion-...,MARMLS,A11801653,N,Y,26.098809,-80.383006,https://www.redfin.com/FL/Weston/1504-Passion-...,Luis Vacca,Sarah Real Estate Group LLC,None,None,Maurizio Bettoli,"Bettoli Real Estate, LLC",None,None,4,blue
4,PAST SALE,2025-07-02,Condo/Co-op,401 N Birch Rd #507,Fort Lauderdale,FL,33304.0,775000.0,1.0,1.5,Gale Residences,794.0,NaN,2018.0,NaN,976.0,1091.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/401-...,Beaches MLS,F10507491,N,Y,26.128325,-80.106224,https://www.redfin.com/FL/Fort-Lauderdale/401-...,None,None,None,None,None,None,None,None,5,blue
5,PAST SALE,2025-07-02,Condo/Co-op,2880 NE 14th Street Cswy #804,Pompano Beach,FL,33062.0,700000.0,3.0,3.0,The Pointe,1900.0,NaN,1999.0,NaN,368.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/2880-N...,Beaches MLS,F10441085,N,Y,26.248695,-80.091423,https://www.redfin.com/FL/Pompano-Beach/2880-N...,Stuart Goodwin,Century 21 Hansen Realty Inc,None,None,Eslin Guice,Icon Realty Advisors LLC,None,None,6,blue
6,PAST SALE,2025-07-02,Condo/Co-op,1750 S Ocean Ln #317,Fort Lauderdale,FL,33316.0,490000.0,2.0,2.0,Lago Mar Place,1158.0,NaN,1962.0,NaN,423.0,1102.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1750...,Beaches MLS,F10434356,N,Y,26.100057,-80.106975,https://www.redfin.com/FL/Fort-Lauderdale/1750...,Rosina Lenardson,Lago Mar Realty Inc,None,None,Steven Bate,Rock and Roll Real Estate,None,None,7,blue
7,PAST SALE,2025-07-02,Condo/Co-op,1945 S Ocean Dr #1512,Hallandale Beach,FL,33009.0,485000.0,1.0,1.5,OCEAN MARINE YACHT CLUB C,995.0,NaN,2007.0,NaN,487.0,1500.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/194...,MARMLS,A11811365,N,Y,25.981318,-80.120771,https://www.redfin.com/FL/Hallandale-Beach/194...,Yure Contreras,"Brickell Realty Group, LLC",Juan Ramirez,"Brickell Realty Group, LLC",Mariana Martynov,Golden Keys Inc,None,None,8,blue
8,PAST SALE,2025-07-01,Condo/Co-op,100 N Federal Hwy #1520,Fort Lauderdale,FL,33301.0,482000.0,2.0,2.0,The Waverly At Las Olas,1325.0,NaN,2005.0,NaN,364.0,1284.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/100-...,

In [36]:
%store -r map_box_api_key

In [37]:
map_box_api_key

'pk.eyJ1IjoidHJkZGF0YSIsImEiOiJjamc2bTc2YmUxY2F3MnZxZGh2amR2MTY5In0.QlOWqB-yQNrNlXD0KQ9IvQ'

In [38]:
import folium
from branca.element import Template, MacroElement

# ——— Your title & caption HTML ———
title_html = f'''
  <h3 align="center" style="font-size:16px"><b>Recent {county} County Condo Sales</b></h3>
'''
caption_html = f'''
  <p align="center" style="font-size:13px"><i>{week_starting} — {week_ending}</i></p>
'''

# ——— Base map ———
m = folium.Map(
    location=df[['LATITUDE','LONGITUDE']].mean().to_list(),
    zoom_start=9,
    tiles=None
)

# ——— Tile layers ———
folium.TileLayer('OpenStreetMap', name='OSM', control=False).add_to(m)
folium.TileLayer(
    tiles=(
        "https://api.mapbox.com/styles/v1/"
        "mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox", name="Streets", control=False
).add_to(m)

# ——— Add your markers ———
for _, row in df_top_ten.iterrows():
    lat, lon = row['LATITUDE'], row['LONGITUDE']
    popup = folium.Popup(popup_html(row), max_width=400)

    if row['COLOR'] == 'blue':
        # Regular sale: blue circle
        folium.CircleMarker(
            location=(lat, lon),
            radius=7,
            fill=True,
            fill_color='blue',
            fill_opacity=0.5,
            color=None,
            popup=popup
        ).add_to(m)
    else:
        # Top sale: gold star
        folium.Marker(
            location=(lat, lon),
            popup=popup,
            icon=folium.DivIcon(
                html="""
                <div style="font-size:24px; line-height:24px; color:gold;">
                  &#9733;
                </div>
                """
            )
        ).add_to(m)

# ——— Layer control, title & caption ———
folium.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

# ——— Optional legend via branca ———
legend_html = """
{% macro html(this, kwargs) %}
  <div style="
      position: fixed;
      bottom: 50px; left: 50px;
      background: white; padding: 8px;
      border: 2px solid grey;
      font-size: 12px;
      z-index: 9999;
  ">
    <div style="display:inline-block; font-size:18px; color:gold; margin-right:6px;">&#9733;</div> Top Sale<br>
    <i style="background: blue; width: 10px; height: 10px;
       display: inline-block; margin-right: 6px;"></i> All other sales
  </div>
{% endmacro %}
"""
legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)

# ——— Display ———
m


In [39]:
m.save('index.html')

## Data snagger

In [40]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [41]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-07-04,Condo/Co-op,730 N Ocean Blvd #803,Pompano Beach,FL,33062.0,2100000.0,2.0,2.5,Sabbia Beach,1880.0,NaN,2019.0,NaN,1117.0,2919.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/730-N-...,Beaches MLS,F10513197,N,Y,26.243932,-80.087321,https://www.redfin.com/FL/Pompano-Beach/730-N-...,David Marder,"Compass Florida, LLC",None,None,David Marder,"Compass Florida, LLC",None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [42]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Broward_County_condo_sales_week_ending_07072025


# CREATE TEMPLATE 

In [43]:
muni_set = set(df_top_ten['CITY'])

In [44]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [45]:
df_top_ten.reset_index(inplace=True,drop=True)

In [46]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [47]:
top_sale

'Sabbia Beach at 730 N Ocean Blvd #803 in Pompano Beach'

In [48]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [49]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [50]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-07-04,Condo/Co-op,730 N Ocean Blvd #803,Pompano Beach,FL,33062.0,2100000.0,2.0,2.5,Sabbia Beach,1880.0,NaN,2019.0,NaN,1117.0,2919.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/730-N-...,Beaches MLS,F10513197,N,Y,26.243932,-80.087321,https://www.redfin.com/FL/Pompano-Beach/730-N-...,David Marder,"Compass Florida, LLC",None,None,David Marder,"Compass Florida, LLC",None,None,1,orange,Sabbia Beach at 730 N Ocean Blvd #803 in Pompa...
1,PAST SALE,2025-07-01,Condo/Co-op,2711 S Ocean Dr #2701,Hollywood,FL,33019.0,1740000.0,3.0,3.5,2711 HOLLYWOOD BEACH COND,2088.0,NaN,2009.0,NaN,833.0,2131.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/2711-S-Oce...,MARMLS,A11704214,N,Y,25.996265,-80.117509,https://www.redfin.com/FL/Hollywood/2711-S-Oce...,Jesus Rincon,Brokernation Real Estate Doral,None,None,Vivian Toppe,Borges Realty Group LLC,None,None,2,blue,2711 HOLLYWOOD BEACH COND at 2711 S Ocean Dr #...
2,PAST SALE,2025-07-01,Condo/Co-op,1912 S Ocean Dr Unit 17A,Hallandale Beach,FL,33009.0,1065000.0,2.0,2.5,MALAGA TOWERS CONDO,2305.0,NaN,1975.0,NaN,462.0,1895.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/191...,MARMLS,A11709779,N,Y,25.981695,-80.118894,https://www.redfin.com/FL/Hallandale-Beach/191...,Pamela DiMartino,Coldwell Banker Realty,None,None,Iuliia Zohar,Best of Luxury Realty Corp.,None,None,3,blue,MALAGA TOWERS CONDO at 1912 S Ocean Dr Unit 17...
3,PAST SALE,2025-07-03,Condo/Co-op,1504 Passion Vine Cir Unit 23-4,Weston,FL,33326.0,800000.0,4.0,2.5,MARIPOSA POINTE AT WESTON,2340.0,NaN,2002.0,NaN,342.0,960.0,Sold,NaN,NaN,https://www.redfin.com/FL/Weston/1504-Passion-...,MARMLS,A11801653,N,Y,26.098809,-80.383006,https://www.redfin.com/FL/Weston/1504-Passion-...,Luis Vacca,Sarah Real Estate Group LLC,None,None,Maurizio Bettoli,"Bettoli Real Estate, LLC",None,None,4,blue,MARIPOSA POINTE AT WESTON at 1504 Passion Vine...
4,PAST SALE,2025-07-02,Condo/Co-op,401 N Birch Rd #507,Fort Lauderdale,FL,33304.0,775000.0,1.0,1.5,Gale Residences,794.0,NaN,2018.0,NaN,976.0,1091.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/401-...,Beaches MLS,F10507491,N,Y,26.128325,-80.106224,https://www.redfin.com/FL/Fort-Lauderdale/401-...,None,None,None,None,None,None,None,None,5,blue,Gale Residences at 401 N Birch Rd #507 in Fort...
5,PAST SALE,2025-07-02,Condo/Co-op,2880 NE 14th Street Cswy #804,Pompano Beach,FL,33062.0,700000.0,3.0,3.0,The Pointe,1900.0,NaN,1999.0,NaN,368.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/2880-N...,Beaches MLS,F10441085,N,Y,26.248695,-80.091423,https://www.redfin.com/FL/Pompano-Beach/2880-N...,Stuart Goodwin,Century 21 Hansen Realty Inc,None,None,Eslin Guice,Icon Realty Advisors LLC,None,None,6,blue,The Pointe at 2880 NE 14th Street Cswy #804 in...
6,PAST SALE,2025-07-02,Condo/Co-op,1750 S Ocean Ln #317,Fort Lauderdale,FL,33316.0,490000.0,2.0,2.0,Lago Mar Place,1158.0,NaN,1962.0,NaN,423.0,1102.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1750...,Beaches MLS,F10434356,N,Y,26.100057,-80.106975,https://www.redfin.com/FL/Fort-Lauderdale/1750...,Rosina Lenardson,Lago Mar Realty Inc,None,None,Steven Bate,Rock and Roll Real Estate,None,None,7,blue,Lago Mar Place at 1750 S Ocean Ln #317 in Fort...
7,PAST SALE,2025-07-02,Condo/Co-op,1945 S Ocean Dr #1512,Hallandale Beach,FL,33009.0,485000.0,1.0,1.5,OCEAN MARINE YACHT CLUB C,995.0,NaN,2007.0,NaN,487.0,1500.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/194...,MARMLS,A11811365,N,Y,25.981318,-80.120771,https://www.redfin.com/FL/Hallandale-Beach/194...,Y

In [51]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Sabbia Beach closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $470,000 to $2,100,000
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Hallandale Beach, Hollywood, Fort Lauderdale, Pompano Beach, Weston, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 66 condo sales totaling $21,285,500 million from June 30 to July 6. The previous week, brokers closed 65 condo sales totaling $21,240,150.

Last week’s units sold for an average of $322,508, lower than the $326,772 average price from the previous week. The

In [52]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [53]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [54]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [55]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-07-04,Condo/Co-op,730 N Ocean Blvd #803,Pompano Beach,FL,33062.0,2100000.0,2.0,2.5,Sabbia Beach,1880.0,NaN,2019.0,NaN,1117.0,2919.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/730-N-...,Beaches MLS,F10513197,N,Y,26.243932,-80.087321,https://www.redfin.com/FL/Pompano-Beach/730-N-...,David Marder,"Compass Florida, LLC",None,None,David Marder,"Compass Florida, LLC",None,None,1,orange,Sabbia Beach at 730 N Ocean Blvd #803 in Pompa...
1,PAST SALE,2025-07-01,Condo/Co-op,2711 S Ocean Dr #2701,Hollywood,FL,33019.0,1740000.0,3.0,3.5,2711 HOLLYWOOD BEACH COND,2088.0,NaN,2009.0,NaN,833.0,2131.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/2711-S-Oce...,MARMLS,A11704214,N,Y,25.996265,-80.117509,https://www.redfin.com/FL/Hollywood/2711-S-Oce...,Jesus Rincon,Brokernation Real Estate Doral,None,None,Vivian Toppe,Borges Realty Group LLC,None,None,2,blue,2711 HOLLYWOOD BEACH COND at 2711 S Ocean Dr #...
2,PAST SALE,2025-07-01,Condo/Co-op,1912 S Ocean Dr Unit 17A,Hallandale Beach,FL,33009.0,1065000.0,2.0,2.5,MALAGA TOWERS CONDO,2305.0,NaN,1975.0,NaN,462.0,1895.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/191...,MARMLS,A11709779,N,Y,25.981695,-80.118894,https://www.redfin.com/FL/Hallandale-Beach/191...,Pamela DiMartino,Coldwell Banker Realty,None,None,Iuliia Zohar,Best of Luxury Realty Corp.,None,None,3,blue,MALAGA TOWERS CONDO at 1912 S Ocean Dr Unit 17...
3,PAST SALE,2025-07-03,Condo/Co-op,1504 Passion Vine Cir Unit 23-4,Weston,FL,33326.0,800000.0,4.0,2.5,MARIPOSA POINTE AT WESTON,2340.0,NaN,2002.0,NaN,342.0,960.0,Sold,NaN,NaN,https://www.redfin.com/FL/Weston/1504-Passion-...,MARMLS,A11801653,N,Y,26.098809,-80.383006,https://www.redfin.com/FL/Weston/1504-Passion-...,Luis Vacca,Sarah Real Estate Group LLC,None,None,Maurizio Bettoli,"Bettoli Real Estate, LLC",None,None,4,blue,MARIPOSA POINTE AT WESTON at 1504 Passion Vine...
4,PAST SALE,2025-07-02,Condo/Co-op,401 N Birch Rd #507,Fort Lauderdale,FL,33304.0,775000.0,1.0,1.5,Gale Residences,794.0,NaN,2018.0,NaN,976.0,1091.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/401-...,Beaches MLS,F10507491,N,Y,26.128325,-80.106224,https://www.redfin.com/FL/Fort-Lauderdale/401-...,None,None,None,None,None,None,None,None,5,blue,Gale Residences at 401 N Birch Rd #507 in Fort...
5,PAST SALE,2025-07-02,Condo/Co-op,2880 NE 14th Street Cswy #804,Pompano Beach,FL,33062.0,700000.0,3.0,3.0,The Pointe,1900.0,NaN,1999.0,NaN,368.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/2880-N...,Beaches MLS,F10441085,N,Y,26.248695,-80.091423,https://www.redfin.com/FL/Pompano-Beach/2880-N...,Stuart Goodwin,Century 21 Hansen Realty Inc,None,None,Eslin Guice,Icon Realty Advisors LLC,None,None,6,blue,The Pointe at 2880 NE 14th Street Cswy #804 in...
6,PAST SALE,2025-07-02,Condo/Co-op,1750 S Ocean Ln #317,Fort Lauderdale,FL,33316.0,490000.0,2.0,2.0,Lago Mar Place,1158.0,NaN,1962.0,NaN,423.0,1102.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1750...,Beaches MLS,F10434356,N,Y,26.100057,-80.106975,https://www.redfin.com/FL/Fort-Lauderdale/1750...,Rosina Lenardson,Lago Mar Realty Inc,None,None,Steven Bate,Rock and Roll Real Estate,None,None,7,blue,Lago Mar Place at 1750 S Ocean Ln #317 in Fort...
7,PAST SALE,2025-07-02,Condo/Co-op,1945 S Ocean Dr #1512,Hallandale Beach,FL,33009.0,485000.0,1.0,1.5,OCEAN MARINE YACHT CLUB C,995.0,NaN,2007.0,NaN,487.0,1500.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/194...,MARMLS,A11811365,N,Y,25.981318,-80.120771,https://www.redfin.com/FL/Hallandale-Beach/194...,Y

In [56]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Sabbia Beach closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $470,000 to $2,100,000
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Hallandale Beach, Hollywood, Fort Lauderdale, Pompano Beach, Weston, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 66 condo sales totaling $21,285,500 million from June 30 to July 6. The previous week, brokers closed 65 condo sales totaling $21,240,150.

Last week’s units sold for an average of $322,508, lower than the $326,772 average price from the previous week. The

In [57]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Pompano-Beach/730-N-Ocean-Blvd-33062/unit-803/home/170421007


In [58]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Hollywood/2711-S-Ocean-Dr-33019/unit-2701/home/42082817


In [59]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Fort-Lauderdale/110-N-Federal-Hwy-33301/unit-1403/home/42034955


In [60]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Fort-Lauderdale/100-N-Federal-Hwy-33301/unit-1520/home/42035452


In [61]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Pompano-Beach/730-N-Ocean-Blvd-33062/unit-803/home/170421007


In [62]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Weston/1504-Passion-Vine-Cir-33326/unit-23-4/home/41998755


In [63]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-07-04,Condo/Co-op,730 N Ocean Blvd #803,Pompano Beach,FL,33062.0,2100000.0,2.0,2.5,Sabbia Beach,1880.0,NaN,2019.0,NaN,1117.0,2919.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/730-N-...,Beaches MLS,F10513197,N,Y,26.243932,-80.087321,https://www.redfin.com/FL/Pompano-Beach/730-N-...,David Marder,"Compass Florida, LLC",None,None,David Marder,"Compass Florida, LLC",None,None,1,orange,Sabbia Beach at 730 N Ocean Blvd #803 in Pompa...
1,PAST SALE,2025-07-01,Condo/Co-op,2711 S Ocean Dr #2701,Hollywood,FL,33019.0,1740000.0,3.0,3.5,2711 HOLLYWOOD BEACH COND,2088.0,NaN,2009.0,NaN,833.0,2131.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/2711-S-Oce...,MARMLS,A11704214,N,Y,25.996265,-80.117509,https://www.redfin.com/FL/Hollywood/2711-S-Oce...,Jesus Rincon,Brokernation Real Estate Doral,None,None,Vivian Toppe,Borges Realty Group LLC,None,None,2,blue,2711 HOLLYWOOD BEACH COND at 2711 S Ocean Dr #...
2,PAST SALE,2025-07-01,Condo/Co-op,1912 S Ocean Dr Unit 17A,Hallandale Beach,FL,33009.0,1065000.0,2.0,2.5,MALAGA TOWERS CONDO,2305.0,NaN,1975.0,NaN,462.0,1895.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/191...,MARMLS,A11709779,N,Y,25.981695,-80.118894,https://www.redfin.com/FL/Hallandale-Beach/191...,Pamela DiMartino,Coldwell Banker Realty,None,None,Iuliia Zohar,Best of Luxury Realty Corp.,None,None,3,blue,MALAGA TOWERS CONDO at 1912 S Ocean Dr Unit 17...
3,PAST SALE,2025-07-03,Condo/Co-op,1504 Passion Vine Cir Unit 23-4,Weston,FL,33326.0,800000.0,4.0,2.5,MARIPOSA POINTE AT WESTON,2340.0,NaN,2002.0,NaN,342.0,960.0,Sold,NaN,NaN,https://www.redfin.com/FL/Weston/1504-Passion-...,MARMLS,A11801653,N,Y,26.098809,-80.383006,https://www.redfin.com/FL/Weston/1504-Passion-...,Luis Vacca,Sarah Real Estate Group LLC,None,None,Maurizio Bettoli,"Bettoli Real Estate, LLC",None,None,4,blue,MARIPOSA POINTE AT WESTON at 1504 Passion Vine...
4,PAST SALE,2025-07-02,Condo/Co-op,401 N Birch Rd #507,Fort Lauderdale,FL,33304.0,775000.0,1.0,1.5,Gale Residences,794.0,NaN,2018.0,NaN,976.0,1091.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/401-...,Beaches MLS,F10507491,N,Y,26.128325,-80.106224,https://www.redfin.com/FL/Fort-Lauderdale/401-...,None,None,None,None,None,None,None,None,5,blue,Gale Residences at 401 N Birch Rd #507 in Fort...
5,PAST SALE,2025-07-02,Condo/Co-op,2880 NE 14th Street Cswy #804,Pompano Beach,FL,33062.0,700000.0,3.0,3.0,The Pointe,1900.0,NaN,1999.0,NaN,368.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/2880-N...,Beaches MLS,F10441085,N,Y,26.248695,-80.091423,https://www.redfin.com/FL/Pompano-Beach/2880-N...,Stuart Goodwin,Century 21 Hansen Realty Inc,None,None,Eslin Guice,Icon Realty Advisors LLC,None,None,6,blue,The Pointe at 2880 NE 14th Street Cswy #804 in...
6,PAST SALE,2025-07-02,Condo/Co-op,1750 S Ocean Ln #317,Fort Lauderdale,FL,33316.0,490000.0,2.0,2.0,Lago Mar Place,1158.0,NaN,1962.0,NaN,423.0,1102.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1750...,Beaches MLS,F10434356,N,Y,26.100057,-80.106975,https://www.redfin.com/FL/Fort-Lauderdale/1750...,Rosina Lenardson,Lago Mar Realty Inc,None,None,Steven Bate,Rock and Roll Real Estate,None,None,7,blue,Lago Mar Place at 1750 S Ocean Ln #317 in Fort...
7,PAST SALE,2025-07-02,Condo/Co-op,1945 S Ocean Dr #1512,Hallandale Beach,FL,33009.0,485000.0,1.0,1.5,OCEAN MARINE YACHT CLUB C,995.0,NaN,2007.0,NaN,487.0,1500.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/194...,MARMLS,A11811365,N,Y,25.981318,-80.120771,https://www.redfin.com/FL/Hallandale-Beach/194...,Y

## Time on Market Calculator

In [64]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2025, 1, 4) ## List (Earlier) date
date2 = datetime(2025, 7, 1) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

178


## Clean CSV for Datawrapper Chart

In [65]:
chart_df = df_top_ten

In [66]:
chart_df = chart_df.fillna(" ")

In [67]:
chart_df['LISTING AGENTS'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING AGENTS'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [68]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [69]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "BEDS", 
    "BATHS",
    "SOURCE", 
    "MLS#"
]

In [70]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [71]:
desired_order = [
    'SOLD DATE',
    'LOCATION',
    'ADDRESS',
    'PRICE',
    'SQUARE FEET',
    '$/SQUARE FEET',
    'YEAR BUILT',
    'LISTING AGENTS',
    'BUYING AGENTS'
]

chart_df = chart_df[desired_order]


In [72]:
chart_df = chart_df.rename(columns={'$/SQUARE FEET':'PPSF'})

In [73]:
# Coerce non-numeric to NaN, then you’ll have a float column
chart_df['PPSF'] = pd.to_numeric(chart_df['PPSF'], errors='coerce')

In [74]:
chart_df['SQUARE FEET'] = pd.to_numeric(chart_df['SQUARE FEET'], errors='coerce')

In [75]:
chart_df['PPSF'].astype(float)

0    1117.0
1     833.0
2     462.0
3     342.0
4     976.0
5     368.0
6     423.0
7     487.0
8     364.0
9     388.0
Name: PPSF, dtype: float64

In [76]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['PPSF'] = chart_df['PPSF'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))

In [77]:
csv_date_string = today.strftime("%m_%d_%Y")

In [78]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [79]:
chart_df

,SOLD DATE,LOCATION,ADDRESS,PRICE,SQUARE FEET,PPSF,YEAR BUILT,LISTING AGENTS,BUYING AGENTS
0,2025-07-04,Sabbia Beach,730 N Ocean Blvd #803 in Pompano Beach,"$2,100,000","1,880","$1,117",2019,"David Marder Compass Florida, LLC","David Marder Compass Florida, LLC"
1,2025-07-01,2711 HOLLYWOOD BEACH COND,2711 S Ocean Dr #2701 in Hollywood,"$1,740,000","2,088",$833,2009,Jesus Rincon Brokernation Real Estate Doral,Vivian Toppe Borges Realty Group LLC
2,2025-07-01,MALAGA TOWERS CONDO,1912 S Ocean Dr Unit 17A in Hallandale Beach,"$1,065,000","2,305",$462,1975,Pamela DiMartino Coldwell Banker Realty,Iuliia Zohar Best of Luxury Realty Corp.
3,2025-07-03,MARIPOSA POINTE AT WESTON,1504 Passion Vine Cir Unit 23-4 in Weston,"$800,000","2,340",$342,2002,Luis Vacca Sarah Real Estate Group LLC,"Maurizio Bettoli Bettoli Real Estate, LLC"
4,2025-07-02,Gale Residences,401 N Birch Rd #507 in Fort Lauderdale,"$775,000",794,$976,2018,,
5,2025-07-02,The Pointe,2880 NE 14th Street Cswy #804 in Pompano Beach,"$700,000","1,900",$368,1999,Stuart Goodwin Century 21 Hansen Realty Inc,Eslin Guice Icon Realty Advisors LLC
6,2025-07-02,Lago Mar Place,1750 S Ocean Ln #317 in Fort Lauderdale,"$490,000","1,158",$423,1962,Rosina Lenardson Lago Mar Realty Inc,Steven Bate Rock and Roll Real Estate
7,2025-07-02,OCEAN MARINE YACHT CLUB C,1945 S Ocean Dr #1512 in Hallandale Beach,"$485,000",995,$487,2007,"Yure Contreras Brickell Realty Group, LLC Juan...",Mariana Martynov Golden Keys Inc
8,2025-07-01,The Waverly At Las Olas,100 N Federal Hwy #1520 in Fort Lauderdale,"$482,000","1,325",$364,2005,Adam Curtis LoKation,Joseph Mongillo LoKation
9,2025-07-03,The Waverly,110 N Federal Hwy #1403 in Fort Lauderdale,"$470,000","1,212",$388,2005,Patricia Finley Coldwell Banker Realty Justine...,One Sotheby's Int'l Realty
